In [1]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import seaborn as sns
import pandas as pd
from functools import reduce
#pip install getFamaFrenchFactors
import getFamaFrenchFactors as gff
#pip install fredapi
from fredapi import Fred
import matplotlib.pyplot as plt
from datetime import datetime
import statsmodels.api as sm

### Fama French

In [2]:
data_ff5 = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.csv', skiprows=3)

In [3]:
data_ff5.rename(columns = {'Unnamed: 0':'Date'},inplace=True)

In [4]:
data_ff5['Date']=data_ff5['Date'].apply(str)


In [5]:
data_ff5.head()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,19630701,-0.67,0.02,-0.35,0.03,0.13,0.012
1,19630702,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,19630703,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,19630705,0.40,0.09,-0.28,0.07,-0.30,0.012
4,19630708,-0.63,0.07,-0.20,-0.27,0.06,0.012


In [6]:
data_ff5.reset_index(drop = True, inplace= True)
data_ff5 = data_ff5[(data_ff5.Date > '20200101') & (data_ff5.Date <= '20231231')]
data_ff5['Date']=pd.to_datetime(data_ff5['Date'])
data_ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
14223,2020-01-02,0.86,-0.97,-0.34,0.24,-0.22,0.006
14224,2020-01-03,-0.67,0.30,0.00,-0.14,-0.10,0.006
14225,2020-01-06,0.36,-0.20,-0.55,-0.17,-0.26,0.006
14226,2020-01-07,-0.19,-0.03,-0.25,-0.12,-0.25,0.006
14227,2020-01-08,0.47,-0.17,-0.64,-0.19,-0.17,0.006
...,...,...,...,...,...,...,...
15183,2023-10-25,-1.58,-0.02,0.84,1.50,0.20,0.021
15184,2023-10-26,-1.15,1.19,1.66,-0.76,0.48,0.021
15185,2023-10-27,-0.53,-0.58,-0.57,0.42,-0.96,0.021
15186,2023-10-30,1.15,-0.34,0.28,-0.14,0.17,0.021


In [7]:
df_ffs = data_ff5.set_index('Date')

In [8]:
df_ffs

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2020-01-02,0.86,-0.97,-0.34,0.24,-0.22,0.006
2020-01-03,-0.67,0.30,0.00,-0.14,-0.10,0.006
2020-01-06,0.36,-0.20,-0.55,-0.17,-0.26,0.006
2020-01-07,-0.19,-0.03,-0.25,-0.12,-0.25,0.006
2020-01-08,0.47,-0.17,-0.64,-0.19,-0.17,0.006
...,...,...,...,...,...,...
2023-10-25,-1.58,-0.02,0.84,1.50,0.20,0.021
2023-10-26,-1.15,1.19,1.66,-0.76,0.48,0.021
2023-10-27,-0.53,-0.58,-0.57,0.42,-0.96,0.021


### ADS Index

In [9]:
data_ads = pd.read_excel('ADS Index Most Current Vintage.xlsx')

In [10]:
data_ads

,Unnamed: 0,ADS_Index
0,1960:03:01,-0.572930
1,1960:03:02,-0.621487
2,1960:03:03,-0.666982
3,1960:03:04,-0.709432
4,1960:03:05,-0.748855
...,...,...
23282,2023:11:28,-0.147276
23283,2023:11:29,-0.143545
23284,2023:11:30,-0.140216
23285,2023:12:01,-0.137288


In [11]:
data_ads.rename(columns = {'Unnamed: 0':'Date'},inplace=True)

In [12]:
data_ads['Date'] = data_ads['Date'].str.replace(':', '-')

In [13]:
data_ads = data_ads[(data_ads.Date > '2020-01-01') & (data_ads.Date <= '2023-12-31')]

In [14]:
data_ads['Date'] = pd.to_datetime(data_ads['Date'])

In [15]:
df_ads = data_ads.set_index('Date')

In [16]:
df_ads

,ADS_Index
Date,
2020-01-02,-0.308743
2020-01-03,-0.292342
2020-01-04,-0.276362
2020-01-05,-0.260796
2020-01-06,-0.245729
...,...
2023-11-28,-0.147276
2023-11-29,-0.143545
2023-11-30,-0.140216


### Fred

In [17]:
fred = Fred(api_key='6fff19c7fff2c26131d75747d34452d6')

In [18]:
fred

In [19]:
varList = ['AMERIBOR', 'T10YIE', 'T5YIE', 'BAMLH0A0HYM2','BAMLH0A0HYM2EY', 
               'DGS10', 'DGS1', 'RIFSPPFAAD90NB', 'DCPN3M', 'DCPF1M', 'DCOILWTICO', 
               'DHHNGSP', 'USRECD', 'USRECDM', 'USRECDP'] 

In [20]:
SP500 = fred.get_series('SP500')
SP500.name = 'SP500'
df_fred = SP500

In [21]:
df_fred

2013-12-13    1775.32
2013-12-16    1786.54
2013-12-17    1781.00
2013-12-18    1810.65
2013-12-19    1809.60
               ...   
2023-12-06    4549.34
2023-12-07    4585.59
2023-12-08    4604.37
2023-12-11    4622.44
2023-12-12    4643.70
Name: SP500, Length: 2608, dtype: float64

In [ ]:
# merge data series
for i in range(0, len(varList)):
    data = fred.get_series(varList[i])
    data.name = varList[i]
    df_fred = pd.merge(df_fred, data, left_index=True, right_index=True)

In [ ]:
df_fred

In [ ]:
df_fred.info()

### Merging Data

In [ ]:
DATA_part1 = pd.merge(df_ffs, df_ads, left_index=True, right_index=True)

In [ ]:
DATA = pd.merge(DATA_part1, df_fred,left_index=True, right_index=True)

In [ ]:
DATA.reset_index(inplace=True)
DATA = DATA.rename(columns = {'index':'Date'})

In [ ]:
DATA_part1

### Collecting Lululemon stocks form yfinance

In [ ]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,12,31)

LULU = yf.download('LULU',start_date ,end_date)
LULU.reset_index(inplace=True)

In [ ]:
LULU

In [ ]:
LULU_new = pd.merge(LULU, DATA,on = 'Date',how='inner')

In [ ]:
LULU_new

In [ ]:
# Yesterday's Close Price
LULU_new['Yest_Close']  = LULU_new['Adj Close'].shift(1)

In [ ]:
# Tesla Stock Returns
LULU_new['stock_return'] = (LULU_new['Adj Close']-LULU_new['Yest_Close'])/(LULU_new['Yest_Close'])

In [ ]:
LULU_new

In [ ]:
LULU_new = LULU_new[LULU_new['Yest_Close'].notna()]
LULU_new = LULU_new[LULU_new['Mkt-RF'].notna()]
LULU_new = LULU_new[LULU_new['SMB'].notna()]
LULU_new = LULU_new[LULU_new['HML'].notna()]
LULU_new = LULU_new[LULU_new['RF'].notna()]

### Correlation

In [ ]:
x = LULU_new.iloc[1:, 1:]
y = LULU_new.iloc[:, -2]
corrmat = LULU_new.corr()
top_corr_features = corrmat.index
plt.figure(figsize = (14,10))
hm = sns.heatmap(LULU_new[top_corr_features].corr(), annot = True, cmap = "YlOrRd")

In [ ]:
X = LULU_new[['Mkt-RF', 'SMB', 'HML', 'ADS_Index']]/100
y = LULU_new['stock_return'] - LULU_new['RF']
X = sm.add_constant(X)

ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())

In [ ]:
X

In [ ]:
y

In [ ]:
ff_model.params

In [ ]:
LULU_new.to_csv('INFOP7374_FeatureMart4GS.csv', index=True)